## src:: https://www.kaggle.com/agustinpugliese/ski-resort-data

# Question No 6


In [138]:
# About data
# Collecting the datasets according to the problem was difficult so I have managed to work with this dataset where
# resort booking details are shown.

import pandas as pd
df = pd.read_csv("Ski_resort.csv")

In [139]:
df['Original_Resort'].unique()

array(['Valle Silbador', 'Montaña Panadero', 'Puente Roto', 'Cresta Velo',
       'Ski Comanche', 'Monte Japón'], dtype=object)

In [184]:
# generating the resort name for the data rows

df['resort'] = ["resort"+str(i) for i in range(0,906)]
df['resort'].nunique()

906


# Features used for clustering

type defines type of booking for season, daily,

Bought_in : defines when the booking was done

Acquisition_mode: how they booked

Schedule : How much they stayed on

Amount_of_days: The number of days they staayed for


Total_price : Total cost

In [141]:
features = ['Type', 'Bought_in','Acquisition_mode','Schedule','Amount_of_days', 'Total_price','resort']
df_filtered = df[features]
df_filtered

,Type,Bought_in,Acquisition_mode,Schedule,Amount_of_days,Total_price,resort
0,Season,Regular_season,Package,Extended_Hour,Unlimited,2925.0,resort0
1,Season,Regular_season,On_site,Extended_Hour,Unlimited,2925.0,resort1
2,Season,Outside_season,Online,Extended_Hour,Unlimited,2925.0,resort2
3,Season,Regular_season,On_site,Extended_Hour,Unlimited,2925.0,resort3
4,Season,Regular_season,Online,Extended_Hour,Unlimited,2925.0,resort4
...,...,...,...,...,...,...,...
901,Daily,Regular_season,On_site,Full_day,1,180.0,resort901
902,Red HEROIC,Regular_season,Red HEROIC,Full_day,1,210.0,resort902
903,Daily,Regular_season,Online,Half_day,1,180.0,resort903
904,Daily,Regular_season,Package,Full_day,1,150.0,resort904


"The finalized data.csv is the generated dataset for resorts problem"

In [142]:
# Exporting to csv file
df_filtered.to_csv("finalized_data.csv")

In [143]:
# using labelencoder for encoding the text into number as we cannot with text data type



from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# df_filtered.iloc[:, 0] = le.fit_transform(df_filtered.iloc[:, 0])
df_filtered.iloc[:, 1] = le.fit_transform(df_filtered.iloc[:, 1])
df_filtered.iloc[:, 2] = le.fit_transform(df_filtered.iloc[:, 2])
df_filtered.iloc[:, 3] = le.fit_transform(df_filtered.iloc[:, 3])
df_filtered.iloc[:, 4] = le.fit_transform(df_filtered.iloc[:, 4])
df_filtered.iloc[:, 6] = le.fit_transform(df_filtered.iloc[:, 6])

/home/addy/yes/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [144]:
df_filtered

,Type,Bought_in,Acquisition_mode,Schedule,Amount_of_days,Total_price,resort
0,Season,1,2,0,9,2925.0,0
1,Season,1,0,0,9,2925.0,1
2,Season,0,1,0,9,2925.0,112
3,Season,1,0,0,9,2925.0,223
4,Season,1,1,0,9,2925.0,334
...,...,...,...,...,...,...,...
901,Daily,1,0,1,0,180.0,892
902,Red HEROIC,1,3,1,0,210.0,893
903,Daily,1,1,2,0,180.0,894
904,Daily,1,2,1,0,150.0,895


In [183]:
# generating the pivot table 

resort_df=df_filtered.pivot_table(index='resort',columns='Type',values='Amount_of_days').fillna(0)
resort_df.head(10)


Type,Daily,Red HEROIC,Season
resort,,,
0,0.0,0.0,9.0
1,0.0,0.0,9.0
2,0.0,0.0,9.0
3,0.0,0.0,9.0
4,0.0,0.0,9.0
5,0.0,0.0,9.0
6,0.0,0.0,9.0
7,0.0,0.0,9.0
8,0.0,0.0,9.0


In [146]:
from scipy.sparse import csr_matrix

resort_df_matrix = csr_matrix(resort_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(resort_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [176]:
def find_similar_resort(query_index):
    distances, indices = model_knn.kneighbors(resort_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10)


    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(resort_df.index[query_index]))
        else:
            print('{0}-> {1}'.format(i, resort_df.index[indices.flatten()[i]]))

In [182]:
def find_top_with_min_price(season):
    index = df_filtered[df_filtered['Type']==season]['Total_price'].argmin()
    return df_filtered.iloc[index]['resort']
    
    
    
# getting the index from user
print("----type 0 for season, 1 for daily, and 2 for daily heroic-----")
dict = {0:'Season', 1:'Daily', 2: 'Daily HEROIC'}
try:
    season = dict[int(input("Enter the season for which you want to travel"))]
    print()
    index = find_top_with_min_price(season)
    print(find_similar_resort(index))
    
    
except Exception as e:
    print('retry with valid data ie. 1,2,0')


----type 0 for season, 1 for daily, and 2 for daily heroic-----
Enter the season for which you want to travel1

Recommendations for 424:

1-> 755
2-> 747
3-> 622
4-> 751
5-> 371
6-> 193
7-> 181
8-> 759
9-> 375
None


# Steps:


For  Recommendation I have used the attributes the amount of days and total price.
First of all user inputs the season. 
The system checks for the resort with most minimum price
Then based upon the index its 10 neighbors are calculated using KNeighbors classifier.

# About Accuracy 
The dataset is generated with same resort so it is expected to raise low accuracy.

# About model

The model helps to achieve higher corelation between the dataset as a result different actions can be taken via user without manully going over the data of resorts. The clients can dive into cheking seasonal resort.